In [3]:
# Install version 4.12.0 of Tweepy

pip install tweepy==4.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 101 kB 3.8 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


In [1]:
# Twitter Auth Codes

API_KEY = 'nsADxJ6111MoCcw756rMG1IwT'

API_SECRET = '1vkGHuGRfSlL49ykh17YFNC8EJ7ELGSbHR9OIbvYQs9uGs0Wam'

BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAMUOhwEAAAAAgzVM3S78tDuzij7Vqz7T4nlEL%2BY%3D1W4trict2Og1wCwqeVcUFxz73nQWbpmZcXfvCLObN0RaYeEBdb'

ACCESS_TOKEN = '1576925312952111104-8cxi4WBT9hQ2myMjmNWe30q7a1wEr5'

ACCESS_TOKEN_SECRET = 'eKmzpYbqRO29z4KpuR0PyZBWSSP1lDXYyiXJLcHgy1LAc'

In [2]:
# Importing Tweepy and creating the Client
import tweepy

client = tweepy.Client(bearer_token = BEARER_TOKEN, wait_on_rate_limit = True)

In [3]:
# Importing necessary packages and reading graph data into Pandas Dataframe (Make sure to upload edges_ex.txt to runtime)
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
data = pd.read_csv('edges_ex.txt', sep=" ", header=None, names=['source', 'target'])

In [4]:
# Creating Graph from Pandas Dataframe using Networkx
g = nx.from_pandas_edgelist(data, create_using = nx.DiGraph())
g = g.reverse()
print(nx.info(g))
# nx.draw(g, with_labels = False)
plt.show

DiGraph with 11726 nodes and 18241 edges


<function matplotlib.pyplot.show(*args, **kw)>

In [11]:
# Triad() is used to implement the structure constraint in the paper (Section 3.1)
def triad():
    triads = []
    for u in g:
        u_followers = g.predecessors(u)
        
        u_following = g.successors(u)
        
        
        for w in u_followers:
            w_following = g.successors(w)
            
            for v in w_following:
                l = [x for x in g.successors(u)]
                if v in l:
                    #print(v)
                    tup = (u, v, w)
                    triads.append(tup)
    print(triads)
    print(len(triads))
    return triads
                 
                    

In [12]:
t = triad()

[(15, 17, 16), (16, 17, 15), (17, 16, 15), (24, 27, 22), (24, 22, 23), (24, 27, 23), (24, 25, 23), (24, 22, 25), (24, 27, 25), (24, 23, 25), (24, 22, 26), (24, 27, 26), (24, 25, 26), (24, 22, 27), (24, 23, 27), (24, 25, 27), (22, 24, 23), (22, 26, 23), (22, 27, 23), (22, 27, 24), (22, 24, 25), (22, 26, 25), (22, 27, 25), (22, 24, 26), (22, 27, 26), (22, 24, 27), (22, 26, 27), (26, 24, 22), (26, 27, 22), (26, 24, 23), (26, 22, 23), (26, 27, 23), (26, 25, 23), (26, 24, 25), (26, 22, 25), (26, 27, 25), (26, 24, 27), (26, 22, 27), (26, 25, 27), (27, 24, 22), (27, 26, 22), (27, 24, 23), (27, 22, 23), (27, 26, 23), (27, 25, 23), (27, 22, 24), (27, 23, 24), (27, 25, 24), (27, 24, 25), (27, 22, 25), (27, 26, 25), (27, 23, 25), (27, 24, 26), (27, 22, 26), (27, 25, 26), (23, 22, 24), (23, 27, 24), (23, 25, 24), (23, 24, 25), (23, 22, 25), (23, 26, 25), (23, 27, 25), (23, 24, 27), (23, 22, 27), (23, 26, 27), (23, 25, 27), (25, 24, 23), (25, 22, 23), (25, 26, 23), (25, 27, 23), (25, 22, 24), (25, 

In [32]:
# Importing Twitter ID's file to lookup based on Node numbers (Make sure to upload twitter-2010-ids.csv to runtime)
twitter_ids = pd.read_csv('/content/twitter-2010-ids.csv', header=0)

In [33]:
twitter_ids['twitter_id'][:1].item()

56862681

In [34]:
# Defining Twitter ID lookup function
def twitter_id_lookup(id):
  x = twitter_ids.loc[twitter_ids['node_id'] == id]['twitter_id']
  return x

In [ ]:
# timed_triads() is used to implement Time ordering of links in the paper. (Section 3.2) (Not used in paper so not used in experiments)
def timed_triads(t):
  for idx, i in enumerate(t):

    u = i[0]
    v = i[1]
    w = i[2]

    w_id = twitter_id_lookup(w)
    w_followers = client.get_users_following(id = int(w_id)).data

    u_id = 0
    v_id = 0

    for idy, j in enumerate(w_followers):
      if (u == j.id):
        u_id = idy
      if (v == j.id):
        v_id = idy
  
    if u_id > v_id:
      t.remove(idx)

  return t 
# t1 to t
t2 = timed_triads(t)

In [117]:
# non_rec() function is used to implement the reciprocity of links in the paper (Section 3.3)
def non_rec(v,w):
    w_followers = g.predecessors(w)
    
    if v in [x for x in w_followers]:
        return 0
    return 1

#sim() function is used to implement the similarity measure as described in the paper (Section 3.4)
def sim(u,v):
    u_following = g.successors(u)
    v_following = g.successors(v)
    
    u_following = set([x for x in u_following])
    v_following = set([x for x in v_following])
    
    intersection = len(u_following.intersection(v_following))
    union = len(u_following.union(v_following))
    
    similarity = intersection / union
    
    
    
    return similarity 

In [118]:
# probability_copy() is used to implement the probability that the link is copied. (Section 3.5)
def probability_copy(triad, flag):
    if flag == 0:
      return 1
    
    elif flag == 1:
      non_rec_value = non_rec(triad[1], triad[2])
      return non_rec_value
    
    elif flag == 2:
      sim1_val = sim(triad[0], triad[1])
      sim2_val = sim(triad[2], triad[0])

      return sim1_val * sim2_val

    else:
      non_rec_value = non_rec(triad[1], triad[2])
        
      sim1_val = sim(triad[0], triad[1])
      sim2_val = sim(triad[2], triad[0])
        
      prob = non_rec_value * sim1_val * sim2_val #implicit structure and time ordered
      return prob

In [119]:
# This is used to sum up all the probabilities (Section 3.5)
def sum_prob(triad_list, flag):
    ans_list = [] 
    for e in triad_list:
        ans = 0
        for f in triad_list:
            if e[1] == f[1] and e[2] == f[2]:
                ans += probability_copy(f, flag)
        ans_list.append(ans)
    return ans_list      

In [120]:
# This function implements the Definition 5 of the paper.
def prob_list(t, ans_list, flag):
    p_list = []
    for i in range(len(t)):
        p = 0
        if ans_list[i] == 0:
            p = 0
        else:
            p = probability_copy(t[i], flag) / ans_list[i]
        p_list.append(p)
    return p_list

In [121]:
# This is used to calculate the cf score (Definition 6)
def cf_score(u):
    sum = 0
    for i in range(len(t)):
        if t[i][0] == u:
            sum += pro_list[i]
    return sum

In [122]:
# This is used to calculate the imitation ration of Section 4 (Definition 7)
def imitation_ratio(u):
    cf = cf_score(u)
    followers_len = len([x for x in g.predecessors(u)])
    
    friends_len = len([x for x in g.successors(u)])
    
    if followers_len and friends_len != 0:
      imitation = cf / (followers_len * friends_len)
    else:
      imitation = 0
    
    return imitation

In [123]:
# Early Adopters Score (Definition 8)
def early_adopter_score(u, v):
    u_followers_len = len([x for x in g.predecessors(u)])
    v_followers_len = len([y for y in g.predecessors(v)])
    #print(u_followers_len, v_followers_len)
    early_adopter = imitation_ratio(u) * (u_followers_len / v_followers_len)
    return early_adopter

In [124]:
early_adop_list = []
for tri in t:
  eas = imitation_ratio(tri[0])
  early_adop_list.append((tri[0], eas))

early_adop_list.sort(key=lambda tup: tup[1], reverse = True)

In [125]:
# Displaying the top 10 early adopters
early_adop_list[:10]

[(582, 1.0),
 (596, 1.0),
 (756, 1.0),
 (800, 1.0),
 (930, 1.0),
 (932, 1.0),
 (935, 1.0),
 (970, 1.0),
 (1003, 1.0),
 (1070, 1.0)]

In [126]:
# Variation 1 of the Future Popularity Score (Using actual Early Adopters score)
def future_popularity_score(v):
    sum = 0
    for u in g.predecessors(v):
        sum += early_adopter_score(u,v)
    return sum

In [127]:
# Variation 2 of the Future Popularity Score (Using Early Adopter score by just Imitation ratio)
def future_popularity_score_2(v):
    sum = 0
    for u in g.predecessors(v):
        sum += imitation_ratio(u)
    return sum

In [184]:
# Variation 1 of Future_popularity_score()

# flag = 0 : only triad
# flag = 1 : triad and non-reciprocity
# flag = 2 : triad and similarity
# flag = 3 : triad, non-reciprocity and similarity

flag = 0

ans_list = sum_prob(t,flag)

pro_list = prob_list(t, ans_list, flag)

future_pop_list = []
for tri in g:
       v = tri
       fps = future_popularity_score(v)
       future_pop_list.append((v,fps))

future_pop_list.sort(key=lambda tup: tup[1], reverse = True)

In [129]:
# Displaying Top 10 nodes with highest Future Popularity score
future_pop_list[:10]

[(2201867, 3.0),
 (39759, 3.0),
 (2424, 2.8970041862899003),
 (2075, 2.6126760563380276),
 (23934050, 2.577174930332825),
 (2892, 2.379252985324976),
 (2885, 2.1264074763749927),
 (695, 2.0),
 (2882, 1.7784033234140046),
 (2675, 1.7269527121910284)]

In [130]:
# From the graph inferring nodes with atleast 10 followers
followers_10 = []
for n in g:
  if len(g.in_edges(n)) >= 10:
    id = twitter_id_lookup(n)
    followers_10.append((n, id.item()))

In [131]:
# Number of nodes (users) in the graph with atleast 10 followers
len(followers_10)

252

In [132]:
# Getting the current follower information of the above nodes and and sorting them in descending order
top_ids = []
for id in followers_10[:]:
  user = client.get_user(id = id[1], user_fields=['public_metrics', 'created_at'])
  if user.data == None:
    pass
  else :
     top_ids.append((id[0], id[1], user.data.public_metrics['followers_count'], user.data.created_at))

top_ids.sort(key=lambda tup: tup[3], reverse = True)

In [133]:
# Finding the accounts which were created in the last 2 weeks, 3 weeks and one month
from datetime import date
from dateutil.relativedelta import relativedelta

two_weeks = top_ids[0][3] + relativedelta(days=-14)
three_weeks = top_ids[0][3] + relativedelta(days=-21)
one_month = top_ids[0][3] + relativedelta(months=-1)

recent_date = top_ids[0][3]

weeks_4 = []
weeks_3 = []
weeks_2 = [] 

for id in top_ids:
  if id[3] <= recent_date and id[3] >= one_month:
    weeks_4.append(id)

for id in top_ids:
  if id[3] <= recent_date and id[3] >= three_weeks:
    weeks_3.append(id)

for id in top_ids:
  if id[3] <= recent_date and id[3] >= two_weeks:
    weeks_2.append(id)

print(len(weeks_4))
print(len(weeks_3))
print(len(weeks_2))


44
42
40


In [134]:
# sorting to descending order to find the latest created account
weeks_4.sort(key=lambda tup: tup[2], reverse = True)
weeks_3.sort(key=lambda tup: tup[2], reverse = True)
weeks_2.sort(key=lambda tup: tup[2], reverse = True)

In [135]:
# Creating Datasets for the experiments with atleast 10, 20, 30 followers and created within the last 2 weeks, 3 weeks and one month 
weeks_4_foll_10 = []
weeks_4_foll_20 = []
weeks_4_foll_30 = []

weeks_3_foll_10 = []
weeks_3_foll_20 = []
weeks_3_foll_30 = []

weeks_2_foll_10 = []
weeks_2_foll_20 = []
weeks_2_foll_30 = []

for e in weeks_4:
  if e[2] >= 10:
    weeks_4_foll_10.append(e)
  if e[2] >= 20:
    weeks_4_foll_20.append(e)
  if e[2] >= 30:
    weeks_4_foll_30.append(e)

print(len(weeks_4_foll_10)) #T4 10
print(len(weeks_4_foll_20)) #T4 20
print(len(weeks_4_foll_30)) #T4 30

37
20
11


In [136]:
for e in weeks_3:
  if e[2] >= 10:
    weeks_3_foll_10.append(e)
  if e[2] >= 20:
    weeks_3_foll_20.append(e)
  if e[2] >= 30:
    weeks_3_foll_30.append(e)

print(len(weeks_3_foll_10)) #T3 10
print(len(weeks_3_foll_20)) #T3 20
print(len(weeks_3_foll_30)) #T3 30

36
19
10


In [137]:
for e in weeks_2:
  if e[2] >= 10:
    weeks_2_foll_10.append(e)
  if e[2] >= 20:
    weeks_2_foll_20.append(e)
  if e[2] >= 30:
    weeks_2_foll_30.append(e[0], e[1])

print(len(weeks_2_foll_10)) #T2 10
print(len(weeks_2_foll_20)) #T2 20
print(len(weeks_2_foll_30)) #T2 30

35
19
10


Experiment - 1 (F1 values) 

In [185]:
# Experiment - 1 ; Value : F1 (-) = T2 10 
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.14257703081232495, pvalue=0.4138998093988494)


In [186]:
# Experiment - 1 ; Value : F1 (-) = T2 20 
pop_score = []
pop_actual = []
for i in weeks_2_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.28596491228070176, pvalue=0.23528462978219675)


In [187]:
# Experiment - 1 ; Value : F1 (-) = T2 30 
pop_score = []
pop_actual = []
for i in weeks_2_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.13939393939393938, pvalue=0.7009318849100584)


In [188]:
# Experiment - 1 ; Value : F1 (-) = T3 10 
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.07773487773487774, pvalue=0.6522549180818393)


In [189]:
# Experiment - 1 ; Value : F1 (-) = T3 20 
pop_score = []
pop_actual = []
for i in weeks_3_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.28596491228070176, pvalue=0.23528462978219675)


In [190]:
# Experiment - 1 ; Value : F1 (-) = T3 30 
pop_score = []
pop_actual = []
for i in weeks_3_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.13939393939393938, pvalue=0.7009318849100584)


In [191]:
# Experiment - 1 ; Value : F1 (-) = T4 10 
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.05523944997629207, pvalue=0.7453900694146549)


In [192]:
# Experiment - 1 ; Value : F1 (-) = T4 20 
pop_score = []
pop_actual = []
for i in weeks_4_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.3533834586466165, pvalue=0.12640665077932206)


In [193]:
# Experiment - 1 ; Value : F1 (-) = T4 30 
pop_score = []
pop_actual = []
for i in weeks_4_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.19090909090909092, pvalue=0.5739131682855215)


In [194]:
# flag = 0 : only triad
# flag = 1 : triad and non-reciprocity
# flag = 2 : triad and similarity
# flag = 3 : triad, non-reciprocity and similarity

flag = 1

ans_list = sum_prob(t,flag)

pro_list = prob_list(t, ans_list, flag)

future_pop_list = []
for tri in g:
       v = tri
       fps = future_popularity_score(v)
       future_pop_list.append((v,fps))

future_pop_list.sort(key=lambda tup: tup[1], reverse = True)

In [195]:
# Experiment - 1 ; Value : F1 (r) = T2 10 
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.0288515406162465, pvalue=0.8693193249493479)


In [196]:
# Experiment - 1 ; Value : F1 (r) = T2 20 
pop_score = []
pop_actual = []
for i in weeks_2_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.4245614035087719, pvalue=0.07001518907656061)


In [197]:
# Experiment - 1 ; Value : F1 (r) = T2 30 
pop_score = []
pop_actual = []
for i in weeks_2_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.01818181818181818, pvalue=0.9602404181286243)


In [198]:
# Experiment - 1 ; Value : F1 (r) = T3 10 
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.05534105534105534, pvalue=0.7485335280007503)


In [199]:
# Experiment - 1 ; Value : F1 (r) = T3 20 
pop_score = []
pop_actual = []
for i in weeks_3_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.4245614035087719, pvalue=0.07001518907656061)


In [200]:
# Experiment - 1 ; Value : F1 (r) = T3 30 
pop_score = []
pop_actual = []
for i in weeks_3_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.01818181818181818, pvalue=0.9602404181286243)


In [201]:
# Experiment - 1 ; Value : F1 (r) = T4 10 
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.08274063537221432, pvalue=0.6263620571654593)


In [202]:
# Experiment - 1 ; Value : F1 (r) = T4 20 
pop_score = []
pop_actual = []
for i in weeks_4_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.4842105263157894, pvalue=0.030508493550666586)


In [203]:
# Experiment - 1 ; Value : F1 (r) = T4 30 
pop_score = []
pop_actual = []
for i in weeks_4_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.18181818181818182, pvalue=0.5926152128455)


In [204]:
# flag = 0 : only triad
# flag = 1 : triad and non-reciprocity
# flag = 2 : triad and similarity
# flag = 3 : triad, non-reciprocity and similarity

flag = 2

ans_list = sum_prob(t,flag)

pro_list = prob_list(t, ans_list, flag)

future_pop_list = []
for tri in g:
       v = tri
       fps = future_popularity_score(v)
       future_pop_list.append((v,fps))

future_pop_list.sort(key=lambda tup: tup[1], reverse = True)

In [205]:
# Experiment - 1 ; Value : F1 (s) = T2 10 
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.24565826330532214, pvalue=0.15489353798432268)


In [206]:
# Experiment - 1 ; Value : F1 (s) = T2 20 
pop_score = []
pop_actual = []
for i in weeks_2_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.24912280701754386, pvalue=0.3037090865488368)


In [207]:
# Experiment - 1 ; Value : F1 (s) = T2 30 
pop_score = []
pop_actual = []
for i in weeks_2_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.19999999999999998, pvalue=0.5795840000000001)


In [208]:
# Experiment - 1 ; Value : F1 (s) = T3 10 
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.17606177606177606, pvalue=0.3043541616960705)


In [209]:
# Experiment - 1 ; Value : F1 (s) = T3 20 
pop_score = []
pop_actual = []
for i in weeks_3_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.24912280701754386, pvalue=0.3037090865488368)


In [210]:
# Experiment - 1 ; Value : F1 (s) = T3 30 
pop_score = []
pop_actual = []
for i in weeks_3_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.19999999999999998, pvalue=0.5795840000000001)


In [211]:
# Experiment - 1 ; Value : F1 (s) = T4 10 
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.15765765765765763, pvalue=0.3513751556718612)


In [212]:
# Experiment - 1 ; Value : F1 (s) = T4 20 
pop_score = []
pop_actual = []
for i in weeks_4_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.3218045112781954, pvalue=0.166474348578758)


In [213]:
# Experiment - 1 ; Value : F1 (s) = T4 30 
pop_score = []
pop_actual = []
for i in weeks_4_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.23636363636363636, pvalue=0.4840911622253965)


In [214]:
# flag = 0 : only triad
# flag = 1 : triad and non-reciprocity
# flag = 2 : triad and similarity
# flag = 3 : triad, non-reciprocity and similarity

flag = 3

ans_list = sum_prob(t,flag)

pro_list = prob_list(t, ans_list, flag)

future_pop_list = []
for tri in g:
       v = tri
       fps = future_popularity_score(v)
       future_pop_list.append((v,fps))

future_pop_list.sort(key=lambda tup: tup[1], reverse = True)

In [215]:
# Experiment - 1 ; Value : F1 (rs) = T2 10 
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.2742296918767507, pvalue=0.11089637980496295)


In [216]:
# Experiment - 1 ; Value : F1 (rs) = T2 20 
pop_score = []
pop_actual = []
for i in weeks_2_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.2742296918767507, pvalue=0.11089637980496295)


In [217]:
# Experiment - 1 ; Value : F1 (rs) = T2 30 
pop_score = []
pop_actual = []
for i in weeks_2_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.006060606060606061, pvalue=0.9867429111949892)


In [218]:
# Experiment - 1 ; Value : F1 (rs) = T3 10 
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.20952380952380953, pvalue=0.2200313026496564)


In [219]:
# Experiment - 1 ; Value : F1 (rs) = T3 20 
pop_score = []
pop_actual = []
for i in weeks_3_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.15614035087719297, pvalue=0.5232597112042991)


In [220]:
# Experiment - 1 ; Value : F1 (rs) = T3 30 
pop_score = []
pop_actual = []
for i in weeks_3_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.006060606060606061, pvalue=0.9867429111949892)


In [221]:
# Experiment - 1 ; Value : F1 (rs) = T4 10 
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.2323376007586534, pvalue=0.1664292621183999)


In [222]:
# Experiment - 1 ; Value : F1 (rs) = T4 20 
pop_score = []
pop_actual = []
for i in weeks_4_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.2270676691729323, pvalue=0.3356744310134179)


In [223]:
# Experiment - 1 ; Value : F1 (rs) = T4 30 
pop_score = []
pop_actual = []
for i in weeks_4_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.21818181818181823, pvalue=0.519248247554982)


Experiment - 1 (F2 Values)

In [ ]:
# flag = 0 : only triad
# flag = 1 : triad and non-reciprocity
# flag = 2 : triad and similarity
# flag = 3 : triad, non-reciprocity and similarity

flag = 0

ans_list = sum_prob(t,flag)

pro_list = prob_list(t, ans_list, flag)

future_pop_list = []
for tri in g:
       v = tri
       fps = future_popularity_score_2(v)
       future_pop_list.append((v,fps))

future_pop_list.sort(key=lambda tup: tup[1], reverse = True)

In [152]:
# Experiment - 1 ; Value : F2 (-) = T2 10 
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)



SpearmanrResult(correlation=0.05602240896358543, pvalue=0.7492356117056708)


In [153]:
# Experiment - 1 ; Value : F2 (-) = T2 20 
pop_score = []
pop_actual = []
for i in weeks_2_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.12631578947368421, pvalue=0.6063502472896753)


In [142]:
# Experiment - 1 ; Value : F2 (-) = T2 30 
pop_score = []
pop_actual = []
for i in weeks_2_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.24848484848484845, pvalue=0.48877630451924314)


In [144]:
# Experiment - 1 ; Value : F2 (-) = T3 10 
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.07104247104247104, pvalue=0.680535513039833)


In [145]:
# Experiment - 1 ; Value : F2 (-) = T3 20 
pop_score = []
pop_actual = []
for i in weeks_3_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.12631578947368421, pvalue=0.6063502472896753)


In [146]:
# Experiment - 1 ; Value : F2 (-) = T3 30 
pop_score = []
pop_actual = []
for i in weeks_3_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.24848484848484845, pvalue=0.48877630451924314)


In [147]:
# Experiment - 1 ; Value : F2 (-) = T4 10 
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.0346135609293504, pvalue=0.8388377237900915)


In [149]:
# Experiment - 1 ; Value : F2 (-) = T4 20 
pop_score = []
pop_actual = []
for i in weeks_4_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.045112781954887216, pvalue=0.8502061902084872)


In [151]:
# Experiment - 1 ; Value : F2 (-) = T4 30 
pop_score = []
pop_actual = []
for i in weeks_4_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.15454545454545454, pvalue=0.65003396549758)


In [154]:
# flag = 0 : only triad
# flag = 1 : triad and non-reciprocity
# flag = 2 : triad and similarity
# flag = 3 : triad, non-reciprocity and similarity

flag = 1

ans_list = sum_prob(t,flag)

pro_list = prob_list(t, ans_list, flag)

future_pop_list = []
for tri in g:
       v = tri
       fps = future_popularity_score_2(v)
       future_pop_list.append((v,fps))

future_pop_list.sort(key=lambda tup: tup[1], reverse = True)

In [155]:
# Experiment - 1 ; Value : F2 (r) = T2 10 
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.09075630252100841, pvalue=0.6041135645371766)


In [156]:
# Experiment - 1 ; Value : F2 (r) = T2 20 
pop_score = []
pop_actual = []
for i in weeks_2_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.09473684210526316, pvalue=0.6996571784752554)


In [157]:
# Experiment - 1 ; Value : F2 (r) = T2 30 
pop_score = []
pop_actual = []
for i in weeks_2_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.1515151515151515, pvalue=0.6760651759978538)


In [158]:
# Experiment - 1 ; Value : F2 (r) = T3 10 
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.26126126126126126, pvalue=0.12377587632461824)


In [159]:
# Experiment - 1 ; Value : F2 (r) = T3 20 
pop_score = []
pop_actual = []
for i in weeks_3_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.09473684210526316, pvalue=0.6996571784752554)


In [160]:
# Experiment - 1 ; Value : F2 (r) = T3 30 
pop_score = []
pop_actual = []
for i in weeks_3_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.1515151515151515, pvalue=0.6760651759978538)


In [161]:
# Experiment - 1 ; Value : F2 (r) = T4 10 
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.1837363679468942, pvalue=0.27634567846215485)


In [162]:
# Experiment - 1 ; Value : F2 (r) = T4 20 
pop_score = []
pop_actual = []
for i in weeks_4_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.045112781954887216, pvalue=0.8502061902084872)


In [163]:
# Experiment - 1 ; Value : F2 (r) = T4 30 
pop_score = []
pop_actual = []
for i in weeks_4_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.11818181818181818, pvalue=0.72928477951978)


In [164]:
# flag = 0 : only triad
# flag = 1 : triad and non-reciprocity
# flag = 2 : triad and similarity
# flag = 3 : triad, non-reciprocity and similarity

flag = 2

ans_list = sum_prob(t,flag)

pro_list = prob_list(t, ans_list, flag)

future_pop_list = []
for tri in g:
       v = tri
       fps = future_popularity_score_2(v)
       future_pop_list.append((v,fps))

future_pop_list.sort(key=lambda tup: tup[1], reverse = True)

In [165]:
# Experiment - 1 ; Value : F2 (s) = T2 10 
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.1285714285714286, pvalue=0.461683326105518)


In [166]:
# Experiment - 1 ; Value : F2 (s) = T2 20 
pop_score = []
pop_actual = []
for i in weeks_2_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.06666666666666667, pvalue=0.7862629610688756)


In [167]:
# Experiment - 1 ; Value : F2 (s) = T2 30 
pop_score = []
pop_actual = []
for i in weeks_2_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.29696969696969694, pvalue=0.4047016712701569)


In [168]:
# Experiment - 1 ; Value : F2 (s) = T3 10 
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.058429858429858435, pvalue=0.7349894427583109)


In [169]:
# Experiment - 1 ; Value : F2 (s) = T3 20 
pop_score = []
pop_actual = []
for i in weeks_3_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.06666666666666667, pvalue=0.7862629610688756)


In [170]:
# Experiment - 1 ; Value : F2 (s) = T3 30 
pop_score = []
pop_actual = []
for i in weeks_3_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.29696969696969694, pvalue=0.4047016712701569)


In [171]:
# Experiment - 1 ; Value : F2 (s) = T4 10 
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.17567567567567566, pvalue=0.29832482884689526)


In [172]:
# Experiment - 1 ; Value : F2 (s) = T4 20 
pop_score = []
pop_actual = []
for i in weeks_4_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.0631578947368421, pvalue=0.7913733978200618)


In [173]:
# Experiment - 1 ; Value : F2 (s) = T4 30 
pop_score = []
pop_actual = []
for i in weeks_4_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.2545454545454546, pvalue=0.4500365768888285)


In [174]:
# flag = 0 : only triad
# flag = 1 : triad and non-reciprocity
# flag = 2 : triad and similarity
# flag = 3 : triad, non-reciprocity and similarity

flag = 3

ans_list = sum_prob(t,flag)

pro_list = prob_list(t, ans_list, flag)

future_pop_list = []
for tri in g:
       v = tri
       fps = future_popularity_score_2(v)
       future_pop_list.append((v,fps))

future_pop_list.sort(key=lambda tup: tup[1], reverse = True)

In [175]:
# Experiment - 1 ; Value : F2 (rs) = T2 10 
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.2873949579831933, pvalue=0.0941248234992954)


In [176]:
# Experiment - 1 ; Value : F2 (rs) = T2 20 
pop_score = []
pop_actual = []
for i in weeks_2_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.02456140350877193, pvalue=0.9204976475735005)


In [177]:
# Experiment - 1 ; Value : F2 (rs) = T2 30 
pop_score = []
pop_actual = []
for i in weeks_2_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.1515151515151515, pvalue=0.6760651759978538)


In [178]:
# Experiment - 1 ; Value : F2 (rs) = T3 10 
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.27644787644787644, pvalue=0.10265120634369183)


In [179]:
# Experiment - 1 ; Value : F2 (rs) = T3 20 
pop_score = []
pop_actual = []
for i in weeks_3_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.02456140350877193, pvalue=0.9204976475735005)


In [180]:
# Experiment - 1 ; Value : F2 (rs) = T3 30 
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.27644787644787644, pvalue=0.10265120634369183)


In [181]:
# Experiment - 1 ; Value : F2 (rs) = T4 10 
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.3686581318160265, pvalue=0.02475314946396297)


In [182]:
# Experiment - 1 ; Value : F2 (rs) = T4 20 
pop_score = []
pop_actual = []
for i in weeks_4_foll_20:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=0.02857142857142857, pvalue=0.9048224147367736)


In [183]:
# Experiment - 1 ; Value : F2 (rs) = T4 30 
pop_score = []
pop_actual = []
for i in weeks_4_foll_30:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

r = stats.spearmanr(pop_pred, pop_actual)
print(r)

SpearmanrResult(correlation=-0.28181818181818186, pvalue=0.4011448979604785)


Experiment - 2 (F2 Values)

In [224]:
# flag = 0 : only triad
# flag = 1 : triad and non-reciprocity
# flag = 2 : triad and similarity
# flag = 3 : triad, non-reciprocity and similarity

flag = 0

ans_list = sum_prob(t,flag)

pro_list = prob_list(t, ans_list, flag)

future_pop_list = []
for tri in g:
       v = tri
       fps = future_popularity_score_2(v)
       future_pop_list.append((v,fps))

future_pop_list.sort(key=lambda tup: tup[1], reverse = True)

In [226]:
# Experiment - 2 ; Value : F2 (-) = T2 10  k@10
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 10)

print(ndcg)

0.05434971510328984


In [227]:
# Experiment - 2 ; Value : F2 (-) = T2 10  k@20
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 20)

print(ndcg)

0.08417128518351237


In [228]:
# Experiment - 2 ; Value : F2 (-) = T2 10  k@50
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 50)

print(ndcg)

0.29672304894114787


In [229]:
# Experiment - 2 ; Value : F2 (-) = T3 10  k@10
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 10)

print(ndcg)

0.047646207277327574


In [230]:
# Experiment - 2 ; Value : F2 (-) = T3 10  k@20
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 20)

print(ndcg)

0.07585713927647617


In [231]:
# Experiment - 2 ; Value : F2 (-) = T3 10  k@50
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 50)

print(ndcg)

0.29187037893291706


In [232]:
# Experiment - 2 ; Value : F2 (-) = T4 10  k@10
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 10)

print(ndcg)

0.04442637690633894


In [233]:
# Experiment - 2 ; Value : F2 (-) = T4 10  k@20
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 20)

print(ndcg)

0.07456049545130226


In [234]:
# Experiment - 2 ; Value : F2 (-) = T4 10  k@50
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 50)

print(ndcg)

0.2921083104700848


In [235]:
# flag = 0 : only triad
# flag = 1 : triad and non-reciprocity
# flag = 2 : triad and similarity
# flag = 3 : triad, non-reciprocity and similarity

flag = 1

ans_list = sum_prob(t,flag)

pro_list = prob_list(t, ans_list, flag)

future_pop_list = []
for tri in g:
       v = tri
       fps = future_popularity_score_2(v)
       future_pop_list.append((v,fps))

future_pop_list.sort(key=lambda tup: tup[1], reverse = True)

In [236]:
# Experiment - 2 ; Value : F2 (r) = T2 10  k@10
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 10)

print(ndcg)

0.050965190139520375


In [237]:
# Experiment - 2 ; Value : F2 (r) = T2 10  k@20
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 20)

print(ndcg)

0.08082958816065859


In [238]:
# Experiment - 2 ; Value : F2 (r) = T2 10  k@50
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 50)

print(ndcg)

0.2926316348666398


In [239]:
# Experiment - 2 ; Value : F2 (r) = T3 10  k@10
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 10)

print(ndcg)

0.057027241682622014


In [240]:
# Experiment - 2 ; Value : F2 (r) = T3 10  k@20
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 20)

print(ndcg)

0.08673624272560992


In [241]:
# Experiment - 2 ; Value : F2 (r) = T3 10  k@50
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 50)

print(ndcg)

0.29860561277743736


In [242]:
# Experiment - 2 ; Value : F2 (r) = T4 10  k@10
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 10)

print(ndcg)

0.057012259456768265


In [243]:
# Experiment - 2 ; Value : F2 (r) = T4 10  k@20
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 20)

print(ndcg)

0.08668619715157519


In [244]:
# Experiment - 2 ; Value : F2 (r) = T4 10  k@50
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 50)

print(ndcg)

0.3000830761241163


In [245]:
# flag = 0 : only triad
# flag = 1 : triad and non-reciprocity
# flag = 2 : triad and similarity
# flag = 3 : triad, non-reciprocity and similarity

flag = 2

ans_list = sum_prob(t,flag)

pro_list = prob_list(t, ans_list, flag)

future_pop_list = []
for tri in g:
       v = tri
       fps = future_popularity_score_2(v)
       future_pop_list.append((v,fps))

future_pop_list.sort(key=lambda tup: tup[1], reverse = True)

In [246]:
# Experiment - 2 ; Value : F2 (s) = T2 10  k@10
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 10)

print(ndcg)

0.057723733233501114


In [247]:
# Experiment - 2 ; Value : F2 (s) = T2 10  k@20
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 20)

print(ndcg)

0.08739616026784028


In [248]:
# Experiment - 2 ; Value : F2 (s) = T2 10  k@50
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 50)

print(ndcg)

0.2946662832878651


In [249]:
# Experiment - 2 ; Value : F2 (s) = T3 10  k@10
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 10)

print(ndcg)

0.05745264672932953


In [250]:
# Experiment - 2 ; Value : F2 (s) = T3 10  k@20
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 20)

print(ndcg)

0.08714088589265757


In [251]:
# Experiment - 2 ; Value : F2 (s) = T3 10  k@50
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 50)

print(ndcg)

0.29581634272703666


In [252]:
# Experiment - 2 ; Value : F2 (s) = T4 10  k@10
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 10)

print(ndcg)

0.057666611302564764


In [253]:
# Experiment - 2 ; Value : F2 (s) = T4 10  k@20
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 20)

print(ndcg)

0.08734083746653781


In [254]:
# Experiment - 2 ; Value : F2 (s) = T4 10  k@50
pop_score = []
pop_actual = []
for i in weeks_4_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 50)

print(ndcg)

0.2974936271757825


In [255]:
# flag = 0 : only triad
# flag = 1 : triad and non-reciprocity
# flag = 2 : triad and similarity
# flag = 3 : triad, non-reciprocity and similarity

flag = 3

ans_list = sum_prob(t,flag)

pro_list = prob_list(t, ans_list, flag)

future_pop_list = []
for tri in g:
       v = tri
       fps = future_popularity_score_2(v)
       future_pop_list.append((v,fps))

future_pop_list.sort(key=lambda tup: tup[1], reverse = True)

In [256]:
# Experiment - 2 ; Value : F2 (rs) = T2 10  k@10
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 10)

print(ndcg)

0.056321101425643255


In [257]:
# Experiment - 2 ; Value : F2 (rs) = T2 10  k@20
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 20)

print(ndcg)

0.0860574485278976


In [258]:
# Experiment - 2 ; Value : F2 (rs) = T2 10  k@50
pop_score = []
pop_actual = []
for i in weeks_2_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 50)

print(ndcg)

0.2933327899695489


In [259]:
# Experiment - 2 ; Value : F2 (rs) = T3 10  k@10
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 10)

print(ndcg)

0.05923307086987402


In [260]:
# Experiment - 2 ; Value : F2 (rs) = T3 10  k@20
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 20)

print(ndcg)

0.08881727537392421


In [261]:
# Experiment - 2 ; Value : F2 (rs) = T3 10  k@50
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 50)

print(ndcg)

0.29657754765317573


In [262]:
# Experiment - 2 ; Value : F2 (rs) = T4 10  k@10
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 10)

print(ndcg)

0.05923307086987402


In [263]:
# Experiment - 2 ; Value : F2 (rs) = T4 10  k@20
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 20)

print(ndcg)

0.08881727537392421


In [264]:
# Experiment - 2 ; Value : F2 (rs) = T4 10  k@50
pop_score = []
pop_actual = []
for i in weeks_3_foll_10:
  pop_actual.append(i[0])
  for item in future_pop_list:
     if item[0] == i[0]:
       pop_score.append(item)


pop_score.sort(key=lambda tup: tup[1], reverse = True)
pop_pred = []
for i in pop_score:
  pop_pred.append(i[0])

from sklearn.metrics import ndcg_score
ndcg = ndcg_score([pop_pred], [pop_actual], k = 50)

print(ndcg)

0.29657754765317573
